# グラフ分類のためのGNN学習

このチュートリアルでは、以下のことができるようになる

- DGLが提供するグラフ分類データセットをロードする
- *readout*関数が何をするかを理解する
- グラフのミニバッチを作成して使用する方法を理解する
- GNNベースのグラフ分類モデルを構築する
- DGLが提供するデータセットでモデルをトレーニングおよび評価する

(Time estimate: 18 minutes)

In [1]:
# ライブラリのインポート
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

## GNNを使ったグラフ分類の概要

グラフ分類（または回帰）は、ノードとエッジの特徴量が与えられた単一のグラフのグラフレベルの特性を予測するモデルが必要とされる。分子特性予測はその一例である。

このチュートリアルでは、グラフ分類モデルを学習させる方法を以下の論文の小規模データセット[How Powerful Are Graph Neural Networks](https://arxiv.org/abs/1810.00826)を用いて紹介する。

## データの読み込み




In [2]:
# ノード数が10から500までの10000個のグラフを持つ合成データセットを生成します。
dataset = dgl.data.GINDataset("PROTEINS", self_loop=True)

In [3]:
dataset

Dataset("PROTEINS", num_graphs=1113, save_path=/home/takutoyo/.dgl/PROTEINS_0c2c49a1)

データセットは、ノード特徴量と単一のラベルを持つグラフのセットである。``GINDataset``オブジェクトの``dim_nfeats``および``gclasses``属性には、ノード特徴量の次元とグラフカテゴリの数が含まれる。

In [4]:
print("Node feature dimensionality:", dataset.dim_nfeats)
print("Number of graph categories:", dataset.gclasses)


from dgl.dataloading import GraphDataLoader

Node feature dimensionality: 3
Number of graph categories: 2


## Data Loaderの定義

グラフ分類データセットは通常、グラフのセットとグラフレベルのラベルを含む2つの要素を持つ。画像分類タスクと同様に、データセットが十分に大きい場合、ミニバッチでトレーニングする必要がある。画像分類や自然言語のモデルを学習させる場合、データセットを反復処理するために``DataLoader``を使用する。DGLでは、``GraphDataLoader``を使用できる。

``GraphDataLoader``は、グラフのミニバッチを生成するためのデータローダーである。``GraphDataLoader``は、``DataLoader``と同様に使用されるが、グラフのミニバッチを生成するために``collate``関数を使用する。

``collate``関数は、グラフのリストを入力として受け取り、バッチ化されたグラフを返す。バッチ化されたグラフは、**単一の大きなグラフになる**。各元のグラフは、バッチ化されたグラフのノードおよびエッジのインデックスを保持するため、バッチ化されたグラフのノードおよびエッジの特徴量テンソルは、元のグラフのノードおよびエッジのインデックスに対応する。

[torch.utils.data.sampler](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler)で提供されるさまざまなデータセットサンプラーを使用することもできる。例えば、このチュートリアルでは、トレーニング``GraphDataLoader``とテスト``GraphDataLoader``を作成し、``SubsetRandomSampler``を使用して、データセットのサブセットからのみサンプリングするようにPyTorchに指示する。

In [5]:
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False
)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size=5, drop_last=False
)

``GraphDataLoader``を作成して、その中をイテレートしてみると、どのようなものが得られるか確認できる。

In [6]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

[Graph(num_nodes=215, num_edges=1041,
      ndata_schemes={'attr': Scheme(shape=(3,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={}), tensor([0, 1, 0, 0, 0])]


``dataset``の各要素にはグラフとラベルが含まれているため、``GraphDataLoader``は各イテレーションで2つのオブジェクトを返す。最初の要素はバッチ化されたグラフであり、2番目の要素は単にミニバッチ内の各グラフのカテゴリを表すラベルベクトルである。次に、バッチ化されたグラフについて説明する。

## DGLのバッチ化されたグラフ

各ミニバッチでは、サンプリングされたグラフは``dgl.batch``を介して単一の大きなバッチ化されたグラフに結合される。単一の大きなバッチ化されたグラフは、元のグラフを個別に接続されたコンポーネントとしてマージし、ノードとエッジの特徴量を連結する。この大きなグラフも``DGLGraph``インスタンスである（したがって、[こちら](2_dglgraph.ipynb)のように通常の``DGLGraph``オブジェクトとして扱うことができる）。ただし、各グラフ要素のノード数とエッジ数など、元のグラフを回復するために必要な情報が含まれている。

In [7]:
batched_graph, labels = batch
print(
    "Number of nodes for each graph element in the batch:",
    batched_graph.batch_num_nodes(),
)
print(
    "Number of edges for each graph element in the batch:",
    batched_graph.batch_num_edges(),
)

# 元のグラフ要素をミニバッチから復元する
graphs = dgl.unbatch(batched_graph)
print("The original graphs in the minibatch:")
print(graphs)

Number of nodes for each graph element in the batch: tensor([34, 15, 46, 84, 36])
Number of edges for each graph element in the batch: tensor([156,  71, 228, 408, 178])
The original graphs in the minibatch:
[Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'attr': Scheme(shape=(3,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={}), Graph(num_nodes=15, num_edges=71,
      ndata_schemes={'attr': Scheme(shape=(3,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={}), Graph(num_nodes=46, num_edges=228,
      ndata_schemes={'attr': Scheme(shape=(3,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={}), Graph(num_nodes=84, num_edges=408,
      ndata_schemes={'attr': Scheme(shape=(3,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={}), Graph(num_nodes=36, num_edges=178,
      ndata_schemes={'attr': Scheme(shape=(3,), dtype=

## モデルの定義



このチュートリアルでは、2層の[Graph Convolutional Network](http://tkipf.github.io/graph-convolutional-networks/)（GCN）を構築する。各レイヤーは、隣接情報を集約して新しいノード表現を計算する。もし、:doc:`introduction <1_introduction>`を読んだことがあれば、2つの違いがわかるだろう。

- ここで実施するタスクは、*グラフ全体*のための単一のカテゴリを予測することであり、各ノードについてではない。そのため、すべてのノードとエッジの表現を集約して新しいグラフレベルの表現を計算する必要がある。このようなプロセスは、一般的に*readout*と呼ばれる。最もシンプルなreadout手法は、``dgl.mean_nodes()``を使用してグラフのノード特徴量を平均することである。

- モデルに入力されるグラフは、``GraphDataLoader``によって生成されるバッチ化されたグラフである。DGLが提供するreadout関数は、バッチ化されたグラフを処理できるため、各ミニバッチ要素に対して1つの表現を返す。


In [12]:
from dgl.nn import GraphConv
from dgl.nn import GINConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata["h"] = h
        return dgl.mean_nodes(g, "h")
    
class GIN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GIN, self).__init__()
        self.conv1 = GINConv(nn.Linear(in_feats, h_feats), "sum")
        self.conv2 = GINConv(nn.Linear(h_feats, num_classes), "sum")

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata["h"] = h
        return dgl.mean_nodes(g, "h")

## 学習ループ

学習ループでは、トレーニングセットを``GraphDataLoader``オブジェクトで反復処理し、勾配を計算する。これは画像分類や言語モデリングと同様である。

In [21]:
from tqdm import tqdm 
# Create the model with given dimensions

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = GCN(dataset.dim_nfeats, 16, dataset.gclasses).to(device)
model = GIN(dataset.dim_nfeats, 16, dataset.gclasses).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in tqdm(range(20)):
    for batched_graph, labels in train_dataloader:
        batched_graph, labels = batched_graph.to(device), labels.to(device)
        pred = model(batched_graph, batched_graph.ndata["attr"].float())
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    batched_graph, labels = batched_graph.to(device), labels.to(device)
    pred = model(batched_graph, batched_graph.ndata["attr"].float())
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print("Test accuracy:", num_correct / num_tests)

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:08<00:00,  1.10it/s]

Test accuracy: 0.0


## What’s next

-  See [GIN
   example](https://github.com/dmlc/dgl/tree/master/examples/pytorch/gin)_
   for an end-to-end graph classification model.




In [ ]:
# Thumbnail credits: DGL
# sphinx_gallery_thumbnail_path = '_static/blitz_5_graph_classification.png'